In [ ]:
/* Welcome to the SQL mini project. You will carry out this project partly in
the PHPMyAdmin interface, and partly in Jupyter via a Python connection.

This is Tier 2 of the case study, which means that there'll be less guidance for you about how to setup
your local SQLite connection in PART 2 of the case study. This will make the case study more challenging for you: 
you might need to do some digging, aand revise the Working with Relational Databases in Python chapter in the previous resource.

Otherwise, the questions in the case study are exactly the same as with Tier 1. 

PART 1: PHPMyAdmin
You will complete questions 1-9 below in the PHPMyAdmin interface. 
Log in by pasting the following URL into your browser, and
using the following Username and Password:

URL: https://sql.springboard.com/
Username: student
Password: learn_sql@springboard

The data you need is in the "country_club" database. This database
contains 3 tables:
    i) the "Bookings" table,
    ii) the "Facilities" table, and
    iii) the "Members" table.

In this case study, you'll be asked a series of questions. You can
solve them using the platform, but for the final deliverable,
paste the code for each solution into this script, and upload it
to your GitHub.

Before starting with the questions, feel free to take your time,
exploring the data, and getting acquainted with the 3 tables. */


/* QUESTIONS 
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

SELECT name
FROM Facilities
WHERE membercost > 0 



/* Q2: How many facilities do not charge a fee to members? */

SELECT COUNT(facid)
FROM Facilities
WHERE membercost = 0



/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */

SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE (membercost > 0) AND (membercost < 0.2 * monthlymaintenance)



/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

SELECT *
FROM Facilities
WHERE facid IN (1, 5)



/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

SELECT name, monthlymaintenance,
	(CASE WHEN monthlymaintenance > 100 THEN 'expensive'
	 ELSE 'cheap' END) AS 'status'
FROM Facilities



/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

SELECT firstname, surname, joindate
FROM Members
WHERE joindate = (SELECT max(joindate) FROM Members)



/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

SELECT DISTINCT CONCAT_WS(' ', M.firstname, M.surname) AS Full_name, F.name AS Facility_name
FROM Bookings AS B
LEFT JOIN Members AS M ON B.memid = M.memid
LEFT JOIN Facilities AS F ON B.facid = F.facid
WHERE F.name LIKE 'tennis court%'
ORDER BY Full_name



/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

SELECT F.name AS Facility_name, CONCAT_WS(' ',M.firstname, M.surname) AS Full_name, 
(CASE WHEN M.memid = 0 THEN F.guestcost*B.slots
 ELSE F.membercost*B.slots END) AS cost
FROM Bookings AS B
LEFT JOIN Members AS M ON B.memid = M.memid
LEFT JOIN Facilities AS F ON B.facid = F.facid
WHERE (CASE WHEN M.memid = 0 THEN F.guestcost*B.slots
 ELSE F.membercost*B.slots END) > 30 AND DATE(starttime) = '2012-09-14'
ORDER BY cost DESC



/* Q9: This time, produce the same result as in Q8, but using a subquery. */

SELECT B.Facility_name, CONCAT_WS(' ',M.firstname, M.surname) AS Full_name, B.cost
FROM (
SELECT 
Bookings.facid, F.name AS Facility_name, starttime, Bookings.memid, (CASE WHEN memid = 0 THEN slots*F.guestcost
ELSE slots*F.membercost END) AS cost
FROM Bookings
LEFT JOIN Facilities AS F ON Bookings.facid = F.facid) AS B
LEFT JOIN Members AS M ON B.memid = M.memid
WHERE B.cost > 30 AND DATE(B.starttime) = '2012-09-14'
ORDER BY B.cost DESC



In [1]:
import sqlite3
from sqlite3 import Error
import pandas as pd

In [2]:
con = sqlite3.connect('sqlite_db_pythonsqlite.db')

In [3]:
print(sqlite3.version)

2.6.0


In [4]:
cur = con.cursor()

In [5]:
query1 = 'SELECT * FROM Facilities'
rs = cur.execute(query1)
facilities = pd.DataFrame(rs.fetchall())

facilities.columns = list(map(lambda x: x[0], rs.description))

In [6]:
facilities.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


In [7]:
query2 = 'SELECT * FROM Bookings'
rs = cur.execute(query2)
bookings = pd.DataFrame(rs.fetchall())

bookings.columns = list(map(lambda x: x[0], rs.description))

In [8]:
bookings.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [9]:
query1 = 'SELECT * FROM Members'
rs = cur.execute(query1)
members = pd.DataFrame(rs.fetchall())

members.columns = list(map(lambda x: x[0], rs.description))

In [10]:
members.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


In [11]:
def query(q):
    rs = cur.execute(q)
    df = pd.DataFrame(rs.fetchall())
    df.columns = list(map(lambda x: x[0], rs.description))
    print(df)

In [12]:
q = """WITH revenue AS(
SELECT F.name AS Facility_name, 
SUM(CASE WHEN B.memid=0 THEN B.slots*F.guestcost
     ELSE B.slots*F.membercost END) AS Revenue
FROM Bookings AS B
LEFT JOIN Facilities AS F
ON B.facid = F.facid
GROUP BY F.name
ORDER BY Revenue)

SELECT Facility_name, Revenue
FROM revenue
WHERE Revenue < 1000
"""

query(q)

   Facility_name  Revenue
0   Table Tennis      180
1  Snooker Table      240
2     Pool Table      270


In [13]:
q = """SELECT (M1.surname || ',' || M1.firstname) AS Member, (M2.surname || ',' || M2.firstname) AS Recommender
FROM Members AS M1
Inner JOIN Members AS M2 on M1.memid = M2.recommendedby
ORDER BY Member, Recommender"""
query(q)

               Member              Recommender
0      Bader,Florence         Sarwin,Ramnaresh
1       Baker,Timothy              Coplin,Joan
2      Butters,Gerald          Genting,Matthew
3      Farrell,Jemima            Baker,Timothy
4      Farrell,Jemima             Pinker,David
5     Genting,Matthew         Rumney,Henrietta
6         Jones,David            Jones,Douglas
7     Joplette,Janice               Dare,Nancy
8     Joplette,Janice              Jones,David
9   Purview,Millicent                Hunt,John
10         Rownam,Tim               Boothe,Tim
11       Smith,Darren           Butters,Gerald
12       Smith,Darren          Joplette,Janice
13       Smith,Darren           Mackenzie,Anna
14       Smith,Darren             Owen,Charles
15       Smith,Darren               Smith,Jack
16        Smith,Tracy            Crumpet,Erica
17        Smith,Tracy        Purview,Millicent
18        Smith,Tracy  Worthington-Smyth,Henry
19    Stibbons,Ponder           Bader,Florence
20    Stibbon

In [14]:
q = """SELECT F.name AS Facility_name, 
SUM(CASE WHEN B.memid != 0 THEN B.slots
    ELSE 0 END) AS Total_usage
FROM Bookings AS B
LEFT JOIN Facilities AS F
ON B.facid = F.facid
GROUP BY F.name
ORDER BY Total_usage DESC"""
query(q)

     Facility_name  Total_usage
0  Badminton Court         1086
1   Tennis Court 1          957
2   Massage Room 1          884
3   Tennis Court 2          882
4    Snooker Table          860
5       Pool Table          856
6     Table Tennis          794
7     Squash Court          418
8   Massage Room 2           54


In [15]:
q = """SELECT F.name AS Facility_name, strftime('%m', DATE(B.starttime)) AS month, 
SUM(CASE WHEN B.memid != 0 THEN B.slots
    ELSE 0 END) AS Total_usage
FROM Bookings AS B
LEFT JOIN Facilities AS F
ON B.facid = F.facid
GROUP BY F.name, Month
ORDER BY Total_usage DESC"""""

query(q)

      Facility_name month  Total_usage
0   Badminton Court    09          507
1        Pool Table    09          443
2    Tennis Court 1    09          417
3   Badminton Court    08          414
4    Tennis Court 2    09          414
5     Snooker Table    09          404
6    Massage Room 1    09          402
7      Table Tennis    09          400
8    Tennis Court 2    08          345
9    Tennis Court 1    08          339
10   Massage Room 1    08          316
11    Snooker Table    08          316
12       Pool Table    08          303
13     Table Tennis    08          296
14   Tennis Court 1    07          201
15     Squash Court    08          184
16     Squash Court    09          184
17   Massage Room 1    07          166
18  Badminton Court    07          165
19    Snooker Table    07          140
20   Tennis Court 2    07          123
21       Pool Table    07          110
22     Table Tennis    07           98
23     Squash Court    07           50
24   Massage Room 2    09